preliminary analysis for llama3 extraction data

In [ ]:
%load_ext autoreload
%autoreload 2

import json

import pandas as pd

from yiutils.project_utils import find_project_root


# params

In [2]:
proj_root = find_project_root("justfile")
print(proj_root)

data_dir = proj_root / "data"
assert data_dir.exists()

output_dir = data_dir / "intermediate" / "llm-results-aggregated"
output_dir.mkdir(parents=True, exist_ok=True)

llm_results_dir = data_dir / "intermediate" / "llm-results"


/Users/ik18445/local-projects/+dmer/+mr-paper-data-extraction/llm-data-extraction


In [ ]:
path_to_results_dir = llm_results_dir / "isb-ai-116732" / "results"
assert path_to_results_dir.exists()


# data loading

In [5]:
json_files = list(path_to_results_dir.glob("*.json"))
print(len(json_files))

json_data = []
for json_file in json_files:
    with open(json_file, "r") as f:
        data = {"data": json.load(f), "filename": str(json_file.name)}
        json_data.append(data)

raw_results_df = pd.concat(
    [
        pd.DataFrame(data["data"]).assign(filename=data["filename"])
        for data in json_data
    ],
).reset_index(drop=True)
raw_results_df.info()
raw_results_df


70
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   pmid                7000 non-null   object
 1   ab                  7000 non-null   object
 2   pub_date            7000 non-null   object
 3   title               7000 non-null   object
 4   journal_issn        6942 non-null   object
 5   journal             7000 non-null   object
 6   author_affil        6932 non-null   object
 7   metadata            6901 non-null   object
 8   results             7000 non-null   object
 9   resultsinformation  6999 non-null   object
 10  metainformation     498 non-null    object
 11  exposures           99 non-null     object
 12  outcomes            99 non-null     object
 13  methods             104 non-null    object
 14  filename            7000 non-null   object
 15  population          83 non-null     object
dtypes: object(16)
memory 

,pmid,ab,pub_date,title,journal_issn,journal,author_affil,metadata,results,resultsinformation,metainformation,exposures,outcomes,methods,filename,population
0,37845420,Previous observational studies focused on the ...,2023-05-27,Causal relationship between coffee intake and ...,1476-5640,European journal of clinical nutrition,"Department of Pain Medicine, The First Affilia...","{'exposures': [{'id': '1', 'trait': 'coffee in...","[{'exposure': 'coffee intake', 'outcome': 'mig...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_21.json,NaN
1,37845373,Previous studies have observed relationships b...,2022-10-04,Assessing the causal relationship between immu...,1617-4623,Molecular genetics and genomics : MGG,"Department of Outpatient and Emergency, First ...","{'exposures': [{'id': '1', 'trait': 'CD20 on I...","[{'exposure': 'CD20 on IgD+ CD38- naïve', 'out...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_21.json,NaN
2,37844535,Based on the findings of current observational...,2023-07-29,Causal relationship between asthma and inflamm...,1527-3288,Heart & lung : the journal of critical care,"Department of General Surgery, The Third Hospi...","{'exposures': [{'id': '1', 'trait': 'Asthma', ...","[{'exposure': 'Asthma', 'outcome': 'Crohn's di...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_21.json,NaN
3,37842613,Previous researches have demonstrated a connec...,2023-08-24,Psychiatric disorders and cardiovascular disea...,2405-8440,Heliyon,Shandong University of Traditional Chinese Med...,"{'exposures': [{'id': '1', 'trait': 'Attention...",[{'exposure': 'Attention deficit hyperactivity...,"{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_21.json,NaN
4,37840498,The role of metabolic syndrome (MetS) on demen...,2023-11-14,Genetic Insights into the Risk of Metabolic Sy...,1875-8908,Journal of Alzheimer's disease : JAD,"Department of Neurosurgery, West China Hospita...","{'exposures': [{'id': '1', 'trait': 'Metabolic...","[{'exposure': 'MetS and its five components', ...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_21.json,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,27351487,Observational studies have reported an associa...,2016-01-19,Obesity and Multiple Sclerosis: A Mendelian Ra...,1549-1676,PLoS medicine,"Department of Epidemiology, Biostatistics and ...","{'exposures': [{'id': '1', 'trait': 'Body mass...","[{'exposure': 'Genetically increased BMI', 'ou...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_59.json,NaN
6996,27342221,Mendelian randomization (MR) studies typically...,2016-03-30,Selecting instruments for Mendelian randomizat...,1464-3685,International journal of epidemiology,"Institute of Cardiovascular Science, Universit...","{'exposures': [], 'outcomes': [], 'methods': [...",[],"{'error': 'No results provided in abstract', '...",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_59.json,NaN
6997,27338949,"Impaired kidney function, as measured by reduc...",2015-10-21,Mendelian Randomisation study of the influence...,2045-2322,Scientific reports,Department of Non-communicable Disease Epidemi...,"{'exposures': [{'id': '1', 'trait': 'Estimated...","[{'exposure': 'eGFR', 'outcome': 'CHD risk', '...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_59.json,NaN
6998,27336604,Observational studies have associated adiposit...,2016-01-29,Mendelian randomisation analysis strongly impl...,1532-1827,British journal of cancer,"Division of Genetics and Epidemiology, The Ins...","{'exposures': [{'id': '1', 'trait': 'Body mass...","[{'exposure': 'BMI', 'outcome': 'Colorectal ca...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_59.json,NaN


In [6]:
output_path = output_dir / "llama3" / "raw_results.json"
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, "w") as f:
    raw_results_df.to_json(f, orient="records", indent=2)


# Process results

In [9]:
results_df = raw_results_df[
    [
        "pmid",
        "ab",
        "title",
        "metadata",
        "results",
    ]
]

results_df.info()
results_df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pmid      7000 non-null   object
 1   ab        7000 non-null   object
 2   title     7000 non-null   object
 3   metadata  6901 non-null   object
 4   results   7000 non-null   object
dtypes: object(5)
memory usage: 273.6+ KB


,pmid,ab,title,metadata,results
0,37845420,Previous observational studies focused on the ...,Causal relationship between coffee intake and ...,"{'exposures': [{'id': '1', 'trait': 'coffee in...","[{'exposure': 'coffee intake', 'outcome': 'mig..."
1,37845373,Previous studies have observed relationships b...,Assessing the causal relationship between immu...,"{'exposures': [{'id': '1', 'trait': 'CD20 on I...","[{'exposure': 'CD20 on IgD+ CD38- naïve', 'out..."
2,37844535,Based on the findings of current observational...,Causal relationship between asthma and inflamm...,"{'exposures': [{'id': '1', 'trait': 'Asthma', ...","[{'exposure': 'Asthma', 'outcome': 'Crohn's di..."
3,37842613,Previous researches have demonstrated a connec...,Psychiatric disorders and cardiovascular disea...,"{'exposures': [{'id': '1', 'trait': 'Attention...",[{'exposure': 'Attention deficit hyperactivity...
4,37840498,The role of metabolic syndrome (MetS) on demen...,Genetic Insights into the Risk of Metabolic Sy...,"{'exposures': [{'id': '1', 'trait': 'Metabolic...","[{'exposure': 'MetS and its five components', ..."
...,...,...,...,...,...
6995,27351487,Observational studies have reported an associa...,Obesity and Multiple Sclerosis: A Mendelian Ra...,"{'exposures': [{'id': '1', 'trait': 'Body mass...","[{'exposure': 'Genetically increased BMI', 'ou..."
6996,27342221,Mendelian randomization (MR) studies typically...,Selecting instruments for Mendelian randomizat...,"{'exposures': [], 'outcomes': [], 'methods': [...",[]
6997,27338949,"Impaired kidney function, as measured by reduc...",Mendelian Randomisation study of the influence...,"{'exposures': [{'id': '1', 'trait': 'Estimated...","[{'exposure': 'eGFR', 'outcome': 'CHD risk', '..."
6998,27336604,Observational studies have associated adiposit...,Mendelian randomisation analysis strongly impl...,"{'exposures': [{'id': '1', 'trait': 'Body mass...","[{'exposure': 'BMI', 'outcome': 'Colorectal ca..."


In [10]:
output_path = output_dir / "llama3" / "processed_results.json"

with open(output_path, "w") as f:
    results_df.to_json(f, orient="records", indent=2)


# basic diagnostics

In [ ]:
# num missing
print(raw_results_df["pmid"].isna().sum(), raw_results_df.shape[0])
# num duplicates
print(
    raw_results_df["pmid"].drop_duplicates().shape[0], raw_results_df["pmid"].shape[0]
)


0 7000
7000 7000


# results summary

# weird examples

In [ ]:
weird_df = raw_results_df[~raw_results_df["exposures"].isna()]
weird_df.info()
weird_df


<class 'pandas.core.frame.DataFrame'>
Index: 99 entries, 88 to 6943
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   pmid                99 non-null     object
 1   ab                  99 non-null     object
 2   pub_date            99 non-null     object
 3   title               99 non-null     object
 4   journal_issn        99 non-null     object
 5   journal             99 non-null     object
 6   author_affil        94 non-null     object
 7   metadata            0 non-null      object
 8   results             99 non-null     object
 9   resultsinformation  99 non-null     object
 10  metainformation     99 non-null     object
 11  exposures           99 non-null     object
 12  outcomes            99 non-null     object
 13  methods             99 non-null     object
 14  filename            99 non-null     object
 15  population          77 non-null     object
dtypes: object(16)
memory usage: 13

,pmid,ab,pub_date,title,journal_issn,journal,author_affil,metadata,results,resultsinformation,metainformation,exposures,outcomes,methods,filename,population
88,37975615,Although a large number of existing studies ha...,2023-10-10,Exploring genetic associations between vitilig...,1600-0625,Experimental dermatology,"Department of Dermatology, The Second Affiliat...",NaN,[],{'error': 'No significant causal associations ...,{'error': 'Some methods do not match those lis...,"[{'id': '1', 'trait': 'Vitiligo', 'category': ...","[{'id': '1', 'trait': 'Depression', 'category'...","[{'id': '1', 'method': 'Inverse-variance weigh...",mr_extract_llama3_sample_array_18.json,NaN
186,37651993,Non-linear Mendelian randomization is an exten...,2022-12-12,Violation of the Constant Genetic Effect Assum...,1423-0062,Human heredity,"MRC Biostatistics Unit, University of Cambridg...",NaN,"[{'exposure': '25-hydroxyvitamin D (25(OH)D)',...","{'error': 'No results provided in abstract', '...",{'error': 'Some methods do not match those lis...,"[{'id': '1', 'trait': '25-hydroxyvitamin D (25...","[{'id': '1', 'trait': 'All-cause mortality', '...","[non-linear Mendelian randomization, residual ...",mr_extract_llama3_sample_array_23.json,[low vitamin D individuals]
203,37390109,Non-linear Mendelian randomization is an exten...,2022-12-26,Relaxing parametric assumptions for non-linear...,1553-7404,PLoS genetics,"MRC Biostatistics Unit, School of Clinical Med...",NaN,"[{'exposure': 'alcohol intake', 'outcome': 'sy...","{'error': 'Some fields are missing', 'explanat...",{'error': 'No information on population is pro...,"[{'id': '1', 'trait': 'Alcohol intake', 'categ...","[{'id': '1', 'trait': 'Systolic blood pressure...","[Non-linear Mendelian randomization, Stratific...",mr_extract_llama3_sample_array_27.json,NaN
268,37316410,Several studies have demonstrated the possible...,2023-01-18,Mendelian Randomisation Study on Association o...,1875-595X,International dental journal,"Department of Oral and Maxillofacial Surgery, ...",NaN,"[{'exposure': 'Lactobacillaceae', 'outcome': '...","{'error': None, 'explanation': None}",{'error': 'No information on population is pro...,"[{'id': '1', 'trait': 'Gut microbiota', 'categ...","[{'id': '1', 'trait': 'Periodontitis', 'catego...","[two-sample mendelian randomization, inverse v...",mr_extract_llama3_sample_array_27.json,[European ancestry]
437,38502664,Multiple Sclerosis (MS) is an immune-related d...,2023-07-10,Association between multiple sclerosis and can...,1932-6203,PloS one,"VIP Inpatient Department, State Key Laboratory...",NaN,"[{'exposure': 'Multiple Sclerosis (MS)', 'outc...","{'error': None, 'explanation': None}",{'error': 'Some methods do not match those lis...,"[{'id': '1', 'trait': 'Multiple Sclerosis (MS)...","[{'id': '1', 'trait': 'cancer', 'category': 'n...","[two-sample MR method, Inverse variance weight...",mr_extract_llama3_sample_array_7.json,[International Multiple Sclerosis Genetics Con...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6551,38453257,"In the most comprehensive analysis to date, th...",2023-11-15,Multi-biobank summary data Mendelian randomisa...,1399-3003,The European respiratory journal,"The MRC Integrative Epidemiology Unit, Univers...",NaN,"[{'exposure': 'CRP-weighted IL-6 signalling', ...","{'error': 'No results provided in abstract', '...",{'error': 'No information on population is pro...,"[{'id': '1', 'trait': 'CRP-weighted IL-6 signa...","[{'id': '1', 'trait': 'PAH risk', 'category': ...",[two-sample mendelian randomization],mr_extract_llama3_sample_array_8.json,[All available PAH GWAS data]
6675,37576893,[This corrects the article DOI: 10.3389/fonc.2...,2023-07-06,Corrigendum: Diseases of the musculoskeletal s...,2234-943X,Frontiers in oncology,"Department of Radiotherapy, The First Affiliat...",NaN,[],"{'error': 'No results provided in abstract', '...","{'error': 'No exposures, outcomes, or methods ...",[],[],[],mr_extract_llama3_sample_array_24.json,[No 